In [ ]:
# The imports
from dotenv import load_dotenv, dotenv_values
from agents import Agent, Runner, trace, SQLiteSession, function_tool
from agents.mcp import MCPServerStdio
import asyncio
import time
import os
import json
from fastapi import FastAPI, UploadFile, File, Form
from fastapi.responses import JSONResponse, FileResponse, HTMLResponse
from fastapi.middleware.cors import CORSMiddleware
from pathlib import Path
import shutil
import nest_asyncio
import uvicorn
from agents.items import ToolCallOutputItem
from mimetypes import guess_type
from typing import Optional
from contextlib import asynccontextmanager
import threading
import requests
from watchdog.observers import Observer
from watchdog.events import FileSystemEventHandler
from openai import OpenAI
import subprocess
import tempfile
from PyPDF2 import PdfReader
from pptx import Presentation
from PIL import Image
import base64
import csv
import pytz
from datetime import datetime

nest_asyncio.apply()

# Load environment variables
load_dotenv(override=True)

openai_api_key = os.getenv('OPENAI_API_KEY')
groq_api_key = os.getenv('GROQ_API_KEY')
client = OpenAI()

# MCP Server configurations
fetch_params = {"command": "uvx", "args": ["mcp-server-fetch"]}

playwright_params = {
    "command": "npx",
    "args": [
        "@playwright/mcp@latest",
        "--browser", "chrome",
        "--extension",
        "--shared-browser-context",
    ],
    "env": {
        "PLAYWRIGHT_MCP_EXTENSION_TOKEN": "CELelKTMrkrFrc4QoB_dybke36loSalmSoz69QHGiWM"
    }
}

# Global variables
playwright_server = None
web_agent = None
file_agent = None

# Storage setup
STORAGE_ROOT = Path(os.getenv("STORAGE_ROOT", "../../storage")).resolve()
METADATA_FILE = STORAGE_ROOT / os.getenv("METADATA_FILE", "file_metadata.json")
UPLOADS_DIR = STORAGE_ROOT / "uploads"

STORAGE_ROOT.mkdir(parents=True, exist_ok=True)
UPLOADS_DIR.mkdir(exist_ok=True)
if not METADATA_FILE.exists():
    with open(METADATA_FILE, "w") as f:
        json.dump([], f)

def load_metadata():
    if METADATA_FILE.exists():
        with open(METADATA_FILE, "r") as f:
            try:
                data = json.load(f)
                return data if isinstance(data, list) else []
            except json.JSONDecodeError:
                return []
    return []

def save_metadata(data):
    with open(METADATA_FILE, "w") as f:
        json.dump(data, f, indent=4)

def get_current_time():
    tz = pytz.timezone("Asia/Kolkata")
    return datetime.now(tz).strftime("%Y-%m-%d %H:%M:%S")

# Web Agent Tools
@function_tool
def get_credentials(site_name: str):
    """Takes site name as input and gives username and password credentials of the site"""
    creds = dotenv_values(".env")
    username_key = f"{site_name.upper()}_USERNAME"
    password_key = f"{site_name.upper()}_PASSWORD"

    if username_key in creds and password_key in creds:
        return creds[username_key], creds[password_key]
    else:
        raise ValueError(f"Credentials for {site_name}")

@function_tool
def wait_for_user(message: str = "Solve CAPTCHA in the browser and press Enter to continue"):
    """Pause execution until the user confirms."""
    print("Hello")
    time.sleep(4)
    return "Resumed after user solved CAPTCHA"

# File Management Tools
@function_tool
def update_file_tool(file_name: str, new_category: str = None, new_tag: str = None):
    """Updates file metadata and optionally moves file between categories."""
    metadata = load_metadata()
    entry = next((m for m in metadata if m["name"] == file_name), None)

    if not entry:
        return {"error": f"File '{file_name}' not found in metadata."}

    src = Path(entry["path"])
    old_category_dir = src.parent

    if new_category:
        new_category_dir = STORAGE_ROOT / new_category
        new_category_dir.mkdir(parents=True, exist_ok=True)

        dst = new_category_dir / src.name
        if dst.exists():
            return {"error": f"File already exists in destination: {dst}"}

        shutil.move(str(src), dst)
        entry["path"] = str(dst)
        entry["category"] = new_category

        try:
            if old_category_dir.exists() and not any(old_category_dir.iterdir()):
                old_category_dir.rmdir()
        except Exception as e:
            print(f"Warning: Could not delete empty folder {old_category_dir}: {e}")

    if new_tag is not None:
        entry["tag"] = new_tag

    entry["last_accessed"] = get_current_time()
    save_metadata(metadata)

    return {
        "success": True,
        "file": entry["name"],
        "new_category": entry["category"],
        "new_tag": entry["tag"],
        "path": entry["path"]
    }

@function_tool
def add_file_tool(file_path: str, category: str, tag: str = None, name: str = None, overwrite: bool = False, description: str = None):
    """Uploads a file into STORAGE_ROOT/category/tag/ and records metadata."""
    src = Path(file_path).resolve()
    if not src.exists():
        return {"error": f"File not found: {src}"}

    if tag:
        target_dir = STORAGE_ROOT / category / tag
    else:
        target_dir = STORAGE_ROOT / category

    target_dir.mkdir(parents=True, exist_ok=True)

    if name:
        name = Path(name).stem
        final_name = f"{name}{src.suffix}"
    else:
        final_name = src.name

    dst = target_dir / final_name

    if dst.exists() and not overwrite:
        base, ext = dst.stem, dst.suffix
        i = 1
        while True:
            new_name = f"{base}_{i}{ext}"
            new_dst = target_dir / new_name
            if not new_dst.exists():
                dst = new_dst
                final_name = new_name
                break
            i += 1

    shutil.copy2(src, dst)

    metadata = load_metadata()
    entry = {
        "name": final_name,
        "path": str(dst),
        "category": category,
        "tag": tag,
        "description": description if description else "No description provided",
        "last_accessed": get_current_time(),
    }
    metadata.append(entry)
    save_metadata(metadata)

    return {
        "success": True,
        "stored_file": final_name,
        "category": category,
        "tag": tag,
        "description": entry["description"]
    }

@function_tool
def get_files_tool(name: str=None, category: str=None, tag: str=None, query: str=None):
    """Fetches files based on filters OR semantic query."""
    metadata = load_metadata()
    results = []
    if query:
        for entry in metadata:
            if query.lower() in entry.get("description","").lower():
                entry["last_accessed"] = datetime.now().isoformat()
                results.append(entry)
    else:
        for entry in metadata:
            match = True
            if name and entry["name"] != name:
                match = False
            if category and entry["category"] != category:
                match = False
            if tag and entry["tag"] != tag:
                match = False
            if match:
                entry["last_accessed"] = datetime.now().isoformat()
                results.append(entry)
    save_metadata(metadata)
    return {"files": results}

@function_tool
def load_metadata_tool():
    """Returns the current metadata.json content as a list of file entries."""
    return load_metadata()

@function_tool
def delete_files_tool(name: str=None, category: str=None, tag: str=None):
    """Deletes files and updates metadata based on filters."""
    metadata = load_metadata()
    updated_metadata = []
    deleted = []

    for entry in metadata:
        match = True
        if name and entry["name"] != name:
            match = False
        if category and entry["category"] != category:
            match = False
        if tag and entry["tag"] != tag:
            match = False

        if match:
            file_path = Path(entry["path"])
            if file_path.exists():
                try:
                    file_path.unlink()
                except Exception as e:
                    return {"error": f"Could not delete {file_path}: {e}"}
            deleted.append(entry)
        else:
            updated_metadata.append(entry)

    save_metadata(updated_metadata)

    def cleanup_empty_folders(root: Path):
        """Recursively remove empty folders under root, ignoring hidden files."""
        for folder in sorted(root.glob("**/*"), key=lambda x: len(x.parts), reverse=True):
            if folder.is_dir():
                if not any(f for f in folder.iterdir() if not f.name.startswith('.')):
                    try:
                        folder.rmdir()
                    except Exception:
                        pass

    cleanup_empty_folders(STORAGE_ROOT)

    return {
        "deleted": [d["name"] for d in deleted],
        "remaining_files": [f["name"] for f in updated_metadata]
    }

@function_tool
def read_contents(file_path: str, max_pages: int = 20, max_chars: int = 10000):
    """Reads the contents of a given file and returns a text preview."""
    path = Path(file_path)
    if not path.exists():
        return {"error": f"File not found: {file_path}"}

    text_content = ""

    try:
        if path.suffix.lower() == ".pdf":
            try:
                reader = PdfReader(str(path))
                for i, page in enumerate(reader.pages[:max_pages]):
                    text_content += f"\n--- Page {i+1} ---\n"
                    extracted = page.extract_text()
                    text_content += extracted if extracted else "[No extractable text]\n"
            except Exception as e:
                return {"error": f"PDF reading failed: {e}"}

        elif path.suffix.lower() in [".pptx", ".ppt"]:
            try:
                text_content = ""
                with tempfile.TemporaryDirectory() as tmpdir:
                    tmpdir_path = Path(tmpdir)
                    pdf_path = tmpdir_path / (path.stem + ".pdf")

                    subprocess.run([
                        "/Applications/LibreOffice.app/Contents/MacOS/soffice",
                        "--headless",
                        "--convert-to", "pdf",
                        str(path),
                        "--outdir", str(tmpdir_path)
                    ], check=True)

                    reader = PdfReader(str(pdf_path))
                    for i, page in enumerate(reader.pages[:max_pages]):
                        text_content += f"\n--- Page {i+1} ---\n"
                        text_content += page.extract_text() or ""

            except Exception as e:
                return {"error": f"PowerPoint PDF extraction failed: {e}"}

        elif path.suffix.lower() == ".json":
            try:
                data = json.loads(path.read_text(errors="ignore"))
                text_content = json.dumps(data, indent=2)[:max_chars]
            except Exception as e:
                return {"error": f"JSON reading failed: {e}"}

        elif path.suffix.lower() == ".csv":
            try:
                with open(path, newline="", encoding="utf-8", errors="ignore") as csvfile:
                    reader = csv.reader(csvfile)
                    rows = []
                    for i, row in enumerate(reader):
                        if i >= max_pages:
                            break
                        rows.append(", ".join(row))
                    text_content = "\n".join(rows)[:max_chars]
            except Exception as e:
                return {"error": f"CSV reading failed: {e}"}

        elif path.suffix.lower() in [".jpg", ".jpeg", ".png"]:
            try:
                with open(path, "rb") as img_file:
                    img_bytes = img_file.read()
                    img_b64 = base64.b64encode(img_bytes).decode("utf-8")

                response = client.chat.completions.create(
                    model="gpt-4o-mini",
                    messages=[
                        {"role": "system", "content": "You are an assistant that describes images in plain language."},
                        {"role": "user", "content": f"Describe this image:\n[base64 image data]\n{img_b64}"}
                    ],
                    max_tokens=300
                )

                text_content = response.choices[0].message.content.strip()

            except Exception as e:
                return {"error": f"Image description failed: {e}"}

        else:
            # Try to read as text file
            try:
                text_content = path.read_text(encoding="utf-8", errors="ignore")[:max_chars]
            except Exception as e:
                return {"error": f"Text file reading failed: {e}"}

        if len(text_content) > max_chars:
            text_content = text_content[:max_chars] + "... [truncated]"

        return {
            "file_name": path.name,
            "path": str(path),
            "content_preview": text_content if text_content.strip() else "[No readable content]"
        }

    except Exception as e:
        return {"error": f"Unexpected error: {e}"}

# Create File Reader Agent
file_reader_agent = Agent(
    name="FileReaderAgent",
    instructions=(
        "You are a file classifier. "
        "1. Always read the file content using `read_contents(file_path)` and summarize it in 1-2 sentences. "
        "2. Use `load_metadata_tool()` to inspect existing files' descriptions, categories, and tags. "
        "3. Recommend the most appropriate **category** and **tag** for this file: "
        "   - The **category** should be a broad, high-level classification (e.g., 'reference_materials', 'novels', 'reports') in snake_case, lowercase, no spaces. "
        "   - The **tag** should be a more specific subtopic within that category (e.g., 'compiler_design', 'fiction', 'annual_report') in snake_case, lowercase, no spaces. "
        "   - Reuse an existing category/tag if it already fits the content; create a new one only if necessary. "
        "4. Only output the recommended category, tag, and 1-2 sentence description. "
        "   Do not modify metadata.json directly."
    ),
    tools=[read_contents, load_metadata_tool],
    model="gpt-4o-mini"
)

reader_tool = file_reader_agent.as_tool(
    tool_name="file_reader",
    tool_description="Reads a file, generates a short description, and suggests the most appropriate category and tag for it."
)

# Create File Manager Agent
file_agent = Agent(
    name="FileManagerAgent",
    instructions=(
        "You are a file manager agent. "
        "Your main responsibility is to manage files: add, update, fetch metadata, and delete files. "
        "\n\n"
        "For file additions: "
        "- Always keep the original 'name' field exactly as the uploaded filename. "
        "- By default, call the `file_reader` tool to get the recommended description, category, and tag. "
        "- If the user explicitly provides a category and/or tag, use those values instead of calling `file_reader` for them. "
        "- If the user provides a description, use it; otherwise, use the one from `file_reader`. "
        "- Ensure that category and tag values are stored in snake_case (e.g., 'reference_materials', 'compiler_design'). "
        "- Never invent categories, tags, or descriptions yourself. "
        "\n\n"
        "For updating or moving files: "
        "- Only modify category, tag, or description if explicitly requested by the user. "
        "- Do not change the filename unless explicitly instructed. "
        "- Do not reclassify the file with `file_reader` unless the user requests reclassification."
        "\n\n"
        "For fetching files: "
        "- Analyze the metadata's description field to determine relevance. "
        "- Return files with all metadata fields (name, category, tag, path, description). "
        "- Never invent or assume content."
        "\n\n"
        "Always ground all decisions in metadata and outputs from `file_reader`. "
        "Never attempt to classify or describe files on your own."
    ),
    tools=[add_file_tool, get_files_tool, delete_files_tool, update_file_tool, reader_tool],
    model="gpt-4o-mini"
)

# Web Agent instructions
web_instructions = """
You are an autonomous browsing agent.

General Browsing Instructions:
- Use ONE browser tab/page for all navigation. Do not open multiple tabs.
- When navigating to websites, use the full URL including https:// (e.g., https://youtube.com)
- Wait for pages to fully load before taking actions.
- Always accept cookies and dismiss pop-ups (e.g., click "Accept", "Not Now") when prompted.
- Close any unnecessary modals, banners, or pop-ups that appear while browsing.

Login Instructions:
- Use the get_credentials tool to retrieve the correct username and password for the website.
- Enter credentials in their respective fields (do not input both in the same box).
- After filling out username and password click on login button (Button might be such as Submit, Log in, etc.).
- If captcha is present do not click on login button. Call the 'wait_for_user' tool.

Website Navigation & Search Instructions:
- Navigate the internet autonomously to complete the users instructions.
- If one website fails to provide the required content, try alternative approaches or websites until the task is completed.
- Follow the users instructions precisely while searching or interacting with the website content.
- Be specific with URLs - for YouTube, use https://www.youtube.com
"""

# OpenAI Routing Function
def get_routing_decision(user_input: str):
    """Use OpenAI API to determine which agent to route to"""
    
    system_prompt = """
You are a routing system that decides which specialized agent should handle user requests.

Classification Rules:
- investigator: Any request involving websites, web browsing, online login, web search, scraping, or internet-based tasks
  Examples: "login vtop", "search google", "browse youtube", "scrape website", "check social media"
  
- FileManagerAgent: Any request involving local file/folder operations 
  Examples: "organize files", "delete documents", "move photos", "search local files", file uploads

Input Processing Rules:
FOR INVESTIGATOR:
- Extract ONLY the "User message" part from the input
- Look for the line that starts with "User message:" and extract everything after it
- Ignore all file metadata, uploaded files info, and system context

FOR FileManagerAgent:
- Pass the COMPLETE original input including all metadata and context

Instructions:
1. Analyze the user's request (look for the "User message:" part)
2. Determine which agent should handle it based on the classification rules
3. Return a JSON response with exactly this format:
{
    "selected_agent": "investigator" or "FileManagerAgent",
    "input": "the appropriate input for the selected agent"
}
If no user input is present default fallback value for selected_agent is FileManagerAgent and input is Store all the files.
Make sure the User input is present in the input section of the JSON
Always return valid JSON. Do not include any other text or explanations outside the JSON.
"""

    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_input}
            ],
            temperature=0.1,
            max_tokens=1000
        )
        
        routing_response = response.choices[0].message.content.strip()
        print(routing_response)
        return json.loads(routing_response)
        
    except Exception as e:
        print(f"Error in routing decision: {e}")
        # Fallback to file agent for safety
        return {
            "selected_agent": "FileManagerAgent",
            "input": "Store the files"
        }

# Main Agent Runner Function
def run_agent_sync(user_input: str, uploaded_files: list[UploadFile] = File(None)):
    """Run the agent routing system using OpenAI for routing decisions"""
    from agents import Runner
    from agents.items import ToolCallOutputItem
    
    # Get routing decision from OpenAI
    routing_decision = get_routing_decision(user_input)
    selected_agent = routing_decision["selected_agent"]
    agent_input = routing_decision["input"]
    
    print(f"Routing to: {selected_agent}")  # Debug log
    
    # Route to appropriate agent based on decision
    if selected_agent == "investigator":
        try:
            loop = asyncio.get_running_loop()
            result = loop.run_until_complete(Runner.run(web_agent, agent_input))
        except RuntimeError:
            result = asyncio.run(Runner.run(web_agent, agent_input))
    elif selected_agent == "FileManagerAgent":

        file_paths = []

        # Save uploaded files
        if uploaded_files:
            for uploaded_file in uploaded_files:
                file_path = UPLOADS_DIR / Path(uploaded_file.filename).name
                with open(file_path, "wb") as f:
                    shutil.copyfileobj(uploaded_file.file, f)
                file_paths.append(file_path)
        
        metadata = load_metadata()
        metadata_json = json.dumps(metadata, indent=2)

        user_input = f"Here is the current file metadata (name, category, tag, description, path):\n{metadata_json}\n\n"
        
        if agent_input:
            user_input += f"User message: {agent_input}\n"
        if file_paths:
            user_input += "Files uploaded:\n" + "\n".join(str(p) for p in file_paths)

        try:
            loop = asyncio.get_running_loop()
            result = loop.run_until_complete(Runner.run(file_agent, user_input))
            
        except RuntimeError:
            result = asyncio.run(Runner.run(file_agent, agent_input))
    else:
        return f"Unknown agent: {selected_agent}", []
    
    # Extract files to show (existing logic)
    files_to_show = []
    for item in result.new_items:
        if isinstance(item, ToolCallOutputItem) and isinstance(item.output, dict):
            if "files" in item.output:
                files_to_show.extend([Path(f["path"]) for f in item.output["files"] if "path" in f])

    return result.final_output, files_to_show

def fetch_files_for_api(name: str=None, category: str=None, tag: str=None):
    """Fetches a file asked by the user and sends it back to him."""
    metadata = load_metadata()
    results = []
    for entry in metadata:
        match = True
        if name and entry["name"] != name:
            match = False
        if category and entry["category"] != category:
            match = False
        if tag and entry["tag"] != tag:
            match = False
        if match:
            results.append(entry)
    return {
        "files": [
            {"name": f["name"], "category": f["category"], "tag": f.get("tag"), "path": f["path"]}
            for f in results
        ]
    }

# Watchdog Setup
DOWNLOADS_DIR = Path.home() / "Downloads"
API_URL = "http://127.0.0.1:8000/chat"
processed_files = set()

def wait_for_file_ready(file_path: Path, timeout=20, poll_interval=0.5):
    start_time = time.time()
    last_size = -1
    while time.time() - start_time < timeout:
        if not file_path.exists():
            time.sleep(poll_interval)
            continue
        current_size = file_path.stat().st_size
        if current_size == last_size and current_size > 0:
            return True
        last_size = current_size
        time.sleep(poll_interval)
    return False

class DownloadHandler(FileSystemEventHandler):
    def on_created(self, event):
        if event.is_directory:
            return
        file_path = Path(event.src_path)

        if file_path.suffix in [".crdownload", ".part", ".tmp"]:
            return

        if file_path in processed_files:
            return
        processed_files.add(file_path)

        print(f"[NEW FILE DETECTED] {file_path}")

        if not wait_for_file_ready(file_path):
            print(f"Warning: File not ready: {file_path}")
            return

        try:
            with open(file_path, "rb") as f:
                files = {"uploaded_files": (file_path.name, f)}
                response = requests.post(API_URL, files=files)

            if response.status_code == 200:
                resp_json = response.json()
                print(f"[AGENT RESPONSE] {resp_json.get('chat_reply', '')}")
                try:
                    file_path.unlink()
                    print(f"Deleted original from Downloads: {file_path}")
                except FileNotFoundError:
                    print(f"Warning: File already deleted: {file_path}")
            else:
                print(f"Warning: API call failed: {response.status_code} {response.text}")

        except Exception as e:
            print(f"[ERROR running agent] {e}")

def start_download_monitor():
    if not DOWNLOADS_DIR.exists():
        print(f"Downloads folder not found: {DOWNLOADS_DIR}")
        return None

    event_handler = DownloadHandler()
    observer = Observer()
    observer.schedule(event_handler, str(DOWNLOADS_DIR), recursive=False)
    observer.start()
    print(f"Monitoring {DOWNLOADS_DIR} for new downloads...")

    def _keep_alive():
        try:
            while True:
                time.sleep(2)
        except KeyboardInterrupt:
            observer.stop()
        observer.join()

    thread = threading.Thread(target=_keep_alive, daemon=True)
    thread.start()
    return observer

# FastAPI Application
@asynccontextmanager
async def lifespan(app: FastAPI):
    # Startup logic
    global playwright_server, web_agent
    
    playwright_server = MCPServerStdio(
        params=playwright_params, 
        client_session_timeout_seconds=60
    )
    await playwright_server.connect()
    
    # Create the web_agent with connected MCP server
    web_agent = Agent(
        name="investigator",
        instructions=web_instructions, 
        model="gpt-5-mini",
        mcp_servers=[playwright_server],
        tools=[get_credentials, wait_for_user]
    )
    
    yield  # This is where the app runs
    
    # Shutdown logic
    if playwright_server:
        await playwright_server.close()

app = FastAPI(title="File Manager Agent", lifespan=lifespan)
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],       
    allow_credentials=True,
    allow_methods=["*"],       
    allow_headers=["*"], 
)

@app.post("/chat")
async def chat_endpoint(
    message: Optional[str] = Form(None),
    uploaded_files: list[UploadFile] = File(None)
):
    if(message==None):
        message=""

    
    file_paths = []
    metadata = load_metadata()
    metadata_json = json.dumps(metadata, indent=2)
    # Save uploaded files
    if uploaded_files:
        for uploaded_file in uploaded_files:
            file_path = UPLOADS_DIR / Path(uploaded_file.filename).name
            with open(file_path, "wb") as f:
                shutil.copyfileobj(uploaded_file.file, f)
            file_paths.append(file_path)

    # Run agent
    reply, files_to_show = run_agent_sync(message, uploaded_files)

    # Prepare clickable URLs
    files_info = []
    for f in files_to_show:
        meta = next((m for m in metadata if Path(m["path"]) == f), None)
        if meta:
            category = meta.get("category")
            tag = meta.get("tag")
            files_info.append({
                "name": meta["name"],
                "category": category,
                "tag": tag,
                "url": f"/download/{meta['name']}?category={category}&tag={tag}"
            })

    return JSONResponse({
        "chat_reply": reply,
        "files": files_info
    })

@app.get("/files")
def get_files():
    try:
        with open(METADATA_FILE, "r") as f:
            data = json.load(f)
        return {"files": data}
    except Exception as e:
        return JSONResponse(status_code=500, content={"error": str(e)})

@app.get("/download/{file_name}")
async def download_file(file_name: str, category: str=None, tag: str=None):
    files = fetch_files_for_api(name=file_name, category=category, tag=tag).get("files", [])
    if not files:
        return JSONResponse({"error": "File not found"}, status_code=404)
    
    file_path = Path(files[0]["path"])
    if not file_path.exists():
        return JSONResponse({"error": "File not found on disk"}, status_code=404)

    return FileResponse(path=file_path, filename=files[0]["name"])

@app.get("/", response_class=HTMLResponse)
async def home():
    return """
    <!DOCTYPE html>
<html lang="en">
<head>
<meta charset="UTF-8">
<title>File Manager Agent</title>
<style>
  body { font-family: Arial, sans-serif; padding: 20px; }
  .chat { margin-bottom: 10px; }
  .file-links { margin-top: 10px; }
  .uploaded-list { margin-top: 10px; color: green; }
</style>
</head>
<body>

<h2>🤖 File Manager Agent</h2>

<form id="chatForm">
  <div class="chat">
    <label>Message:</label><br>
    <input type="text" id="message" size="50">
  </div>
  <div class="chat">
    <label>Upload Files (optional):</label><br>
    <input type="file" id="file" multiple>
  </div>
  <button type="submit">Send</button>
</form>

<h3>Uploaded Files:</h3>
<div id="uploadedFiles" class="uploaded-list"></div>

<h3>Reply:</h3>
<div id="reply"></div>

<h3>Files:</h3>
<div id="files" class="file-links"></div>

<script>
const form = document.getElementById('chatForm');
const fileInput = document.getElementById('file');
const uploadedFilesDiv = document.getElementById('uploadedFiles');

fileInput.addEventListener('change', () => {
  uploadedFilesDiv.innerHTML = '';
  if (fileInput.files.length > 0) {
    uploadedFilesDiv.innerHTML = `<strong>${fileInput.files.length} file(s) selected:</strong><br>`;
    for (let i = 0; i < fileInput.files.length; i++) {
      uploadedFilesDiv.innerHTML += `• ${fileInput.files[i].name}<br>`;
    }
  } else {
    uploadedFilesDiv.innerHTML = 'No files selected';
  }
});

form.addEventListener('submit', async (e) => {
  e.preventDefault();

  const formData = new FormData();
  formData.append('message', document.getElementById('message').value);

  if (fileInput.files.length > 0) {
    for (let i = 0; i < fileInput.files.length; i++) {
      formData.append('uploaded_files', fileInput.files[i]);
    }
  }

  const response = await fetch('/chat', {
    method: 'POST',
    body: formData
  });

  const data = await response.json();
  document.getElementById('reply').innerText = data.chat_reply;

  const filesDiv = document.getElementById('files');
  filesDiv.innerHTML = '';
  if (data.files && data.files.length > 0) {
    data.files.forEach(f => {
      const link = document.createElement('a');
      link.href = f.url;
      link.innerText = f.name;
      link.target = '_blank';
      filesDiv.appendChild(link);
      filesDiv.appendChild(document.createElement('br'));
    });
  }

  // Clear file input, uploaded files list, and optionally text
  fileInput.value = "";
  uploadedFilesDiv.innerHTML = 'No files selected';
  document.getElementById('message').value = "";
});
</script>

</body>
</html>
    """

# Start monitoring downloads
observer = start_download_monitor()

# Main execution
async def main():
    config = uvicorn.Config(app=app, host="0.0.0.0", port=8000, log_level="info")
    server = uvicorn.Server(config)
    await server.serve()

if __name__ == "__main__":
    asyncio.run(main())